In [1]:
import requests
import json
from queue import Queue
from threading import Thread
from time import sleep
import pandas as pd
import re
import logging
import coloredlogs
from datetime import datetime
import sys
import random

sys.tracebacklimit = 0
logging.raiseExceptions = False

In [3]:
with open('lazada_categories.json', 'r') as f:
  categories = json.loads(
      f.read())['data']['resultValue']['categoriesLpMultiFloor']['data']

lst = []
for category in categories:
  for subCategory in category['level2TabList']:
    url = subCategory['categoryUrl']
    print(url)
    lst.append(url)

print(len(lst))

//www.lazada.vn/dien-thoai-di-dong
//www.lazada.vn/may-tinh-bang
//www.lazada.vn/laptop
//www.lazada.vn/may-tinh-de-ban-va-phu-kien
//www.lazada.vn/am-thanh
//www.lazada.vn/camera-giam-sat-2
//www.lazada.vn/may-anh-may-quay-phim
//www.lazada.vn/may-quay-phim
//www.lazada.vn/man-hinh-vi-tinh
//www.lazada.vn/man-hinh-may-in
//www.lazada.vn/dong-ho-thong-minh
//www.lazada.vn/dieu-khien-choi-game
//www.lazada.vn/phu-kien-dien-thoai-may-tinh-bang
//www.lazada.vn/thiet-bi-thong-minh
//www.lazada.vn/thiet-bi-so
//www.lazada.vn/phu-kien-may-anh-may-quay-phim
//www.lazada.vn/phu-kien-may-bay-camera
//www.lazada.vn/thiet-bi-luu-tru-2
//www.lazada.vn/phu-kien-may-vi-tinh
//www.lazada.vn/linh-kien-may-tinh
//www.lazada.vn/phu-kien-ong-kinh
//www.lazada.vn/thiet-bi-mang
//www.lazada.vn/phu-kien-may-choi-game
//www.lazada.vn/tv-video-am-thanh-thiet-bi-deo-cong-nghe
//www.lazada.vn/do-gia-dung-nho
//www.lazada.vn/do-gia-dung-lon
//www.lazada.vn/phu-kien-cho-tv
//www.lazada.vn/cham-soc-da-mat
//www.la

In [7]:
int(str('24% OFF').split(' ')[0][:-1])/100

0.24